In [ ]:
import json
import requests
import pandas as pd
import geopandas as gpd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

In [ ]:
deso_url = 'https://geodata.scb.se/geoserver/stat/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=stat%3ADeSO.2018&outputFormat=Geopackage&format_options=charset:UTF-8'
deso = gpd.read_file(deso_url)

In [ ]:
deso = deso[['deso', 'kommun', 'lan', 'kommunnamn', 'lannamn', 'geometry']]

## Utbildningsnivå

In [ ]:
session = requests.Session()
x = open('req_data/pxapi-api_table_UtbSUNBefDesoRegso.json')
query = json.load(x)['queryObj']
url = 'https://api.scb.se/OV0104/v1/doris/sv/ssd/START/UF/UF0506/UF0506YDeso/UtbSUNBefDesoRegso'
query['response']['format'] = 'json'
response = session.post(url, json=query)
response_json = json.loads(response.content)

In [ ]:
df = pd.DataFrame(data = response_json['data'])
values = pd.DataFrame(df['values'].to_list(), columns = ['Antal_utbildningslängd'])
split = pd.DataFrame(df['key'].tolist(), columns = ['DESO', 'Utbildningslängd', 'År'])
split.loc[split.Utbildningslängd == '21', 'Utbildningslängd'] = 'Förgymnasial'
split.loc[split.Utbildningslängd == '3+4', 'Utbildningslängd'] = 'Gymnasial'
split.loc[split.Utbildningslängd == '5', 'Utbildningslängd'] = 'Eftergymnasial < 3 år'
split.loc[split.Utbildningslängd == '6', 'Utbildningslängd'] = 'Eftergymnasial >= 3 år'
split.loc[split.Utbildningslängd == 'US', 'Utbildningslängd'] = 'Uppgift saknas'
df = split.join(values).drop('År', axis = 1)
df = df.pivot(index = 'DESO', columns = 'Utbildningslängd', values = 'Antal_utbildningslängd')
df.columns.name = None
df = df.astype(int)
df = df.divide(df.sum(axis = 1), axis = 0)
deso = pd.merge(deso, df, left_on = 'deso', right_index = True)

## Folkmängd per bakgrund & kön

In [ ]:
session = requests.Session()
x = open('req_data/pxapi-api_table_FolkmDesoBakgrKonN.json')
query = json.load(x)['queryObj']
url = 'https://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101Y/FolkmDesoBakgrKonN'
query['response']['format'] = 'json'
response = session.post(url, json=query)
response_json = json.loads(response.content)

In [ ]:
df = pd.DataFrame(data = response_json['data'])
values = pd.DataFrame(df['values'].to_list(), columns = ['Antal_härkomst_kön'])
split = pd.DataFrame(df['key'].tolist(), columns = ['DESO', 'Härkomst', 'Kön', 'År'])
split.loc[split.Härkomst == '1', 'Härkomst'] = 'Utländsk'
split.loc[split.Härkomst == '2', 'Härkomst'] = 'Svensk'
split.loc[split.Kön == '1', 'Kön'] = 'Man'
split.loc[split.Kön == '2', 'Kön'] = 'Kvinna'
df = split.join(values).drop('År', axis = 1)
df = df.pivot(index = 'DESO', columns = ['Härkomst', 'Kön'], values = 'Antal_härkomst_kön')
df.columns.name = None
df.columns = df.columns.to_flat_index()
df = df.astype(int)
df = df.divide(df.sum(axis = 1), axis = 0)
deso = pd.merge(deso, df, left_on = 'deso', right_index = True)

## Folkmängd per åldersgrupp

In [ ]:
session = requests.Session()
x = open('req_data/pxapi-api_table_FolkmDesoAldKonN.json')
query = json.load(x)['queryObj']
url = 'https://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101Y/FolkmDesoAldKonN'
query['response']['format'] = 'json'
response = session.post(url, json=query)
response_json = json.loads(response.content)

In [ ]:
df = pd.DataFrame(data = response_json['data'])
values = pd.DataFrame(df['values'].to_list(), columns = ['Antal_åldersgrupp'])
split = pd.DataFrame(df['key'].tolist(), columns = ['DESO', 'Åldersgrupp', 'År'])
df = split.join(values).drop('År', axis = 1)
df = df.pivot(index = 'DESO', columns = ['Åldersgrupp'], values = 'Antal_åldersgrupp')
df.columns.name = None
df = df.astype(int)
df = df.divide(df.sum(axis = 1), axis = 0)
deso = pd.merge(deso, df, left_on = 'deso', right_index = True)

## Folkmängd per civilstånd & kön

In [ ]:
session = requests.Session()
x = open('req_data/pxapi-api_table_FolkmDesoCivilKonN.json')
query = json.load(x)['queryObj']
url = 'https://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101Y/FolkmDesoCivilKonN'
query['response']['format'] = 'json'
response = session.post(url, json=query)
response_json = json.loads(response.content)

In [ ]:
df = pd.DataFrame(data = response_json['data'])
values = pd.DataFrame(df['values'].to_list(), columns = ['Antal_civilstånd'])
split = pd.DataFrame(df['key'].tolist(), columns = ['DESO', 'Civilstånd', 'Kön', 'År'])
split.loc[split.Kön == '1', 'Kön'] = 'Man'
split.loc[split.Kön == '2', 'Kön'] = 'Kvinna'
df = split.join(values).drop('År', axis = 1)
df = df.pivot(index = 'DESO', columns = ['Civilstånd', 'Kön'], values = 'Antal_civilstånd')
df.columns.name = None
df.columns = df.columns.to_flat_index()
df = df.astype(int)
df = df.divide(df.sum(axis = 1), axis = 0)
deso = pd.merge(deso, df, left_on = 'deso', right_index = True)

## Folkmängd per hushållstyp

In [ ]:
session = requests.Session()
x = open('req_data/pxapi-api_table_HushallDesoTypN.json')
query = json.load(x)['queryObj']
url = 'https://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101Y/HushallDesoTypN'
query['response']['format'] = 'json'
response = session.post(url, json=query)
response_json = json.loads(response.content)

In [ ]:
df = pd.DataFrame(data = response_json['data'])
values = pd.DataFrame(df['values'].to_list(), columns = ['Antal_HH'])
split = pd.DataFrame(df['key'].tolist(), columns = ['DESO', 'Hushållstyp', 'År'])
df = split.join(values).drop('År', axis = 1)
df = df.pivot(index = 'DESO', columns = ['Hushållstyp'], values = 'Antal_HH')
df.columns.name = None
df = df.astype(int)
df.drop('TOTALT', axis = 1, inplace = True)
df = df.divide(df.sum(axis = 1), axis = 0)
deso = pd.merge(deso, df, left_on = 'deso', right_index = True)

## Personbilar per status

In [ ]:
session = requests.Session()
x = open('req_data/pxapi-api_table_PersBilarDeso.json')
query = json.load(x)['queryObj']
url = 'https://api.scb.se/OV0104/v1/doris/sv/ssd/START/TK/TK1001/TK1001Z/PersBilarDeso'
query['response']['format'] = 'json'
response = session.post(url, json=query)
response_json = json.loads(response.content)

In [ ]:
df = pd.DataFrame(data = response_json['data'])
values = pd.DataFrame(df['values'].to_list(), columns = ['Antal_personbilar'])
split = pd.DataFrame(df['key'].tolist(), columns = ['DESO', 'Status', 'År'])
df = split.join(values).drop('År', axis = 1)
df = df.pivot(index = 'DESO', columns = ['Status'], values = 'Antal_personbilar')
df.columns.name = None
df = df.astype(int)
df.loc[:, ['AVST', 'ITRAF']] = df.loc[:, ['AVST', 'ITRAF']].divide(df.loc[:, ['AVST', 'ITRAF']].sum(axis = 1), axis = 0)
deso = pd.merge(deso, df, left_on = 'deso', right_index = True)

## Antal personer per inkomnstkvartial (i %)

In [ ]:
session = requests.Session()
x = open('req_data/pxapi-api_table_Tab1InkDesoN.json')
query = json.load(x)['queryObj']
url = 'https://api.scb.se/OV0104/v1/doris/sv/ssd/START/HE/HE0110/HE0110I/Tab1InkDesoN'
query['response']['format'] = 'json'
response = session.post(url, json=query)
response_json = json.loads(response.content)

In [ ]:
df = pd.DataFrame(data = response_json['data'])
values = pd.DataFrame(df['values'].to_list(), columns = ['Kvartil_1', 'Kvartil_2', 'Kvartil_3', 'Kvartil_4', 'Medianink', 'Medelink'])
split = pd.DataFrame(df['key'].tolist(), columns = ['DESO', 'Nettoink', 'Kön', 'År'])
df = split.join(values).drop(['År', 'Nettoink', 'Kön'], axis = 1)
df.index = df.DESO
df.drop('DESO', axis = 1, inplace = True)
df = df.astype(float)
df.loc[:, ['Kvartil_1', 'Kvartil_2', 'Kvartil_3', 'Kvartil_4']] = df.loc[:, ['Kvartil_1', 'Kvartil_2', 'Kvartil_3', 'Kvartil_4']].divide(df.loc[:, ['Kvartil_1', 'Kvartil_2', 'Kvartil_3', 'Kvartil_4']].sum(axis = 1), axis = 0)
deso = pd.merge(deso, df, left_on = 'deso', right_index = True)

## Förvärvsarbetande befolkning

In [ ]:
session = requests.Session()
x = open('req_data/pxapi-api_table_BefDeSoSyssN.json')
query = json.load(x)['queryObj']
url = 'https://api.scb.se/OV0104/v1/doris/sv/ssd/START/AM/AM0207/AM0207I/BefDeSoSyssN'
query['response']['format'] = 'json'
response = session.post(url, json=query)
response_json = json.loads(response.content)

In [ ]:
df = pd.DataFrame(data = response_json['data'])
values = pd.DataFrame(df['values'].to_list(), columns = ['Antal_arbetsstatus'])
split = pd.DataFrame(df['key'].tolist(), columns = ['DESO', 'Arbetsstatus', 'Kön', 'År'])
split.loc[split.Kön == '1', 'Kön'] = 'Man'
split.loc[split.Kön == '2', 'Kön'] = 'Kvinna'
split.loc[split.Kön == '1+2', 'Kön'] = 'Båda'
split = split.loc[split.Kön != 'Båda']
df = split.join(values).drop('År', axis = 1)
df = df.pivot(index = 'DESO', columns = ['Arbetsstatus', 'Kön'], values = 'Antal_arbetsstatus')
df.columns.name = None
df.columns = df.columns.to_flat_index()
df = df.astype(int)
df.loc[:, [('FÖRV', 'Man'), ('EJFÖRV', 'Man')]] = df.loc[:, [('FÖRV', 'Man'), ('EJFÖRV', 'Man')]].divide(df[('total', 'Man')], axis = 0)
df.loc[:, [('FÖRV', 'Kvinna'), ('EJFÖRV', 'Kvinna')]] = df.loc[:, [('FÖRV', 'Kvinna'), ('EJFÖRV', 'Kvinna')]].divide(df[('total', 'Kvinna')], axis = 0)
df['TOT_BEF'] = df[[('total', 'Man'), ('total', 'Kvinna')]].sum(axis = 1)
df.drop([('total', 'Man'), ('total', 'Kvinna')], axis = 1, inplace = True)
deso = pd.merge(deso, df, left_on = 'deso', right_index = True)

In [ ]:
deso['BEF_THT'] = deso['TOT_BEF'].divide(deso.area, axis = 0)
deso['FORDON_PP'] = deso['TOT'].divide(deso['TOT_BEF'], axis = 0)

In [ ]:
new_columns = []
for i in deso.columns:
    if type(i) == tuple:
        j = '_'.join(i).rstrip('_')
        new_columns.append(j)
    else:
        new_columns.append(i)
deso.columns = new_columns

In [ ]:
deso.to_file('output/preprocessed_data.gpkg')

In [ ]:
pd.DataFrame(data = deso).to_pickle('output/preprocessed_data.pkl')